In [8]:
import pandas as pd
import scanpy as sc

In [9]:
import os

HC_path = '../multi_region_single_cell/HC//minor_cell_type/'
adata = sc.read_h5ad(f'{HC_path}Ast.h5ad')

In [10]:
cell_types = []
for i in os.listdir(HC_path):
    if '.h5ad' in i:
        cell_types.append(i)
labels  = ['clinical_AD_Label','Pathological_AD_Label'] 
cell_types

['Per.h5ad',
 'Opc.h5ad',
 'Oli.h5ad',
 'Ast.h5ad',
 'Mic.h5ad',
 'SMC.h5ad',
 'End.h5ad',
 'Inh.h5ad',
 'Fib.h5ad',
 'CAM.h5ad',
 'CPEC.h5ad',
 'T.h5ad',
 'Exc.h5ad',
 'Epd.h5ad']

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [12]:
all_genes = adata.var.varnames.tolist()

In [13]:
import numpy as np

def return_ensemble_id(x,df_ensemble):
    try:
        return df_ensemble[df_ensemble['1']==x]['0'].values[0]
    except: 
        return np.nan

def return_clinical_label(x):
    if x['clinical_diagnosis'] == 'AD':
        return 1.0
    elif x['clinical_diagnosis'] == 'NCI':
        return 0.0
    else:
        return 2.0


def return_pthological_label(x):
    if x['pathological_diagnosis'] == 'AD':
        return 1.0
    elif x['pathological_diagnosis'] == 'No AD':
        return 0.0
    else:
        return 2.0

def return_clinical_pthological_label(x):
    if x['clinical_pathological_AD'] == 'AD_with_Plaques':
        return 1.0
    elif x['clinical_pathological_AD'] == 'NCI_with_No_Plaques':
        return 0.0
    else:
        return 2.0

# Evaluation


clinical_data = pd.read_csv('../clinical_single_cell.csv')

df_ensemble = pd.read_csv('../df_ensemble.csv')



gene_excel = pd.ExcelFile('../202141022_Genelist_m6aproject_BED.xlsx')
m6a = pd.read_excel(gene_excel, 'm6A')
m6a = m6a.iloc[2:,]
m6a_genes = m6a['m6A related genes/proteins'].tolist()

mito = pd.read_excel(gene_excel, 'Mitochondria')
mito  = mito.iloc[2:,]
mito_genes = mito['Mitochondria genes'].tolist()


mitophagy = pd.read_excel(gene_excel, 'Mitophagy')
mitophagy = mitophagy.iloc[2:,]
mitophagy_genes = mitophagy['Autophagy_Mitophagy_lysosome'].tolist()

AD = pd.read_excel(gene_excel, 'AD')
AD   = AD.iloc[2:,]
AD_genes = AD["Alzheimer's disease - risk factors"].tolist()


mito = pd.read_excel(gene_excel, 'Mitochondria')
mito  = mito.iloc[2:,]
mito_genes = mito['Mitochondria genes'].tolist()
mito_genes[0:5]

lysosome = pd.read_excel(gene_excel, 'Lysosome')
lysosome  = lysosome.iloc[2:,]
lysosome_genes = lysosome['Lysosomes'].tolist()

genes_set  =  [all_genes,m6a_genes,mito_genes,mitophagy_genes,AD_genes,mito_genes,lysosome_genes]
genes_set_name  =  ['all_genes','m6a_genes','mito_genes','mitophagy_genes','AD_genes','mito_genes','lysosome_genes']


        
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            outputs = model(inputs).squeeze(1)
            preds = torch.sigmoid(outputs) > 0.5  # Convert logits to binary predictions
            correct += (preds == labels.squeeze(1)).sum().item()
            total += labels.size(0)
            Accuracy = 100 * correct / total
    return Accuracy

def train_model(model, train_loader,test_loader, criterion, optimizer, epochs=10):
    model.train()
    best_accuracy = 0
    for epoch in range(epochs):
        total_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            optimizer.zero_grad()
            outputs = model(inputs).squeeze(1)  # Remove extra dimension
            loss = criterion(outputs, labels.squeeze(1).float())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        test_accuracy  = evaluate_model(model, test_loader)
        train_accuracy = evaluate_model(model, train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}, TrainAccuracy: {train_accuracy:.2f}, Test_Accuracy: {test_accuracy:.2f}")
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
    
    return best_accuracy




class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

accuracy_dct = dict()

for i in cell_types:
    print(i)
    adata = sc.read_h5ad(f'{HC_path}{i}')
    temp_data = pd.merge(adata.obs.reset_index(), clinical_data, on =  'projid', how='inner').drop_duplicates()
    temp_data = temp_data.set_index(adata.obs.index)
    adata.obs = temp_data
    adata.obs['clinical_AD_Label'] = adata.obs.apply(return_clinical_label,axis=1)
    adata.obs['Pathological_AD_Label'] = adata.obs.apply(return_pthological_label,axis=1)
    adata.obs['Clinical_Pathological_AD_Label'] = adata.obs.apply(return_clinical_pthological_label,axis=1)
    adata.obs.drop('obsnames', axis=1, inplace=True)
    
    for  j in labels:
        print(j)
        subset_adata = adata[adata.obs[j].isin([0.0,1.0]),:]
        Neg = subset_adata.obs[subset_adata.obs[j] ==0.0]
        Pos = subset_adata.obs[subset_adata.obs[j] ==1.0]
        max_sample = min(len(Neg),len(Pos))
        Neg = subset_adata.obs[subset_adata.obs[j] ==0.0].iloc[0:max_sample,:]
        Pos = subset_adata.obs[subset_adata.obs[j] ==1.0].iloc[0:max_sample,:]
        data = pd.concat([Neg,Pos])
        subset_adata = subset_adata[data.index,:]
        count_matrix_df = pd.DataFrame(
                    subset_adata.X.toarray(),  # Convert sparse matrix to dense array if needed
                    index=subset_adata.obs.index,  # Cell IDs as row indices
                    columns=subset_adata.var.index  # Gene names as column indices
                )
    
        
        for l,k in enumerate(genes_set):
            #print(k)
            keep_genes = list(set(k).intersection(set(count_matrix_df.columns)))
            count_matrix_df = count_matrix_df[keep_genes]
            label_df = pd.DataFrame(subset_adata.obs[j])
            name = genes_set_name[l]


             # ML part
            X = torch.tensor(count_matrix_df.values, dtype=torch.float32)
            y = torch.tensor(label_df.values, dtype=torch.long)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
            X_train, X_test, y_train, y_test = X_train.to(device) , X_test.to(device)  , y_train.to(device) , y_test.to(device)

            dataset_train = TensorDataset(X_train, y_train)
            dataset_test = TensorDataset(X_test, y_test)
            input_size = X.shape[1]
            hidden_size =      X.shape[1]//2
            output_size = 1  # Binary classification
            model = NeuralNet(input_size, hidden_size, output_size).to(device) 
            train_loader = DataLoader(dataset_train, batch_size=4096, shuffle=True)
            test_loader = DataLoader(dataset_test, batch_size=4096, shuffle=False)
            criterion = nn.BCEWithLogitsLoss()
            optimizer = optim.Adam(model.parameters(), lr=0.0001)
            best_test_accuracy = train_model(model, train_loader,test_loader, criterion, optimizer, epochs=1000)

            accuracy_dct[f'{i.split('.')[0]}.{name}.{j}'] = best_test_accuracy
            print(accuracy_dct)
             

Per.h5ad
clinical_AD_Label
Epoch 1/1000, Loss: 0.6944, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 2/1000, Loss: 2.4523, TrainAccuracy: 67.42, Test_Accuracy: 51.28
Epoch 3/1000, Loss: 0.6063, TrainAccuracy: 51.94, Test_Accuracy: 50.00
Epoch 4/1000, Loss: 0.7491, TrainAccuracy: 70.00, Test_Accuracy: 51.28
Epoch 5/1000, Loss: 0.5456, TrainAccuracy: 92.26, Test_Accuracy: 71.79
Epoch 6/1000, Loss: 0.5093, TrainAccuracy: 78.39, Test_Accuracy: 60.26
Epoch 7/1000, Loss: 0.5327, TrainAccuracy: 81.61, Test_Accuracy: 61.54
Epoch 8/1000, Loss: 0.5049, TrainAccuracy: 93.55, Test_Accuracy: 73.08
Epoch 9/1000, Loss: 0.4387, TrainAccuracy: 99.68, Test_Accuracy: 61.54
Epoch 10/1000, Loss: 0.3850, TrainAccuracy: 94.84, Test_Accuracy: 52.56
Epoch 11/1000, Loss: 0.3426, TrainAccuracy: 100.00, Test_Accuracy: 70.51
Epoch 12/1000, Loss: 0.2621, TrainAccuracy: 99.35, Test_Accuracy: 78.21
Epoch 13/1000, Loss: 0.2123, TrainAccuracy: 100.00, Test_Accuracy: 80.77
Epoch 14/1000, Loss: 0.1515, TrainAccuracy: 

/home/anwer/.local/lib/python3.12/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Epoch 54/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 55/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 56/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 57/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 58/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 59/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 60/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 61/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 62/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 63/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 64/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 65/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 66/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 67/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy

/home/anwer/.local/lib/python3.12/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Epoch 48/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 49/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 50/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 51/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 52/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 53/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 54/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 55/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 56/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 57/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 58/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 59/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 60/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 61/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy

...

/home/anwer/.local/lib/python3.12/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Epoch 1/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 2/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 3/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 4/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 5/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 6/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 7/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 8/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 9/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 10/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 11/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 12/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 13/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
Epoch 14/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.00
E

/home/anwer/.local/lib/python3.12/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Epoch 1/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 49.99
Epoch 2/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 49.99
Epoch 3/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 49.99
Epoch 4/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.01
Epoch 5/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.01
Epoch 6/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.01
Epoch 7/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.01
Epoch 8/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.01
Epoch 9/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.01
Epoch 10/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.01
Epoch 11/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.01
Epoch 12/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.01
Epoch 13/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.01
Epoch 14/1000, Loss: 0.6931, TrainAccuracy: 50.00, Test_Accuracy: 50.01
E

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
1000//3

In [ ]:
li = []
for i in x:
    if i == True:
        li.append(1)
    else:
        li.append(0)

In [ ]:
s = 0
for i,j in enumerate(li):
    if y[i] == j:
        s +=1

In [ ]:
s/len(y)

In [ ]:
s

In [15]:
accuracy_dct

{'Per.all_genes.clinical_AD_Label': 80.76923076923077,
 'Per.m6a_genes.clinical_AD_Label': 60.256410256410255,
 'Per.mito_genes.clinical_AD_Label': 50.0,
 'Per.mitophagy_genes.clinical_AD_Label': 50.0,
 'Per.AD_genes.clinical_AD_Label': 50.0,
 'Per.lysosome_genes.clinical_AD_Label': 50.0,
 'Per.all_genes.Pathological_AD_Label': 80.76923076923077,
 'Per.m6a_genes.Pathological_AD_Label': 61.53846153846154,
 'Per.mito_genes.Pathological_AD_Label': 50.0,
 'Per.mitophagy_genes.Pathological_AD_Label': 50.0,
 'Per.AD_genes.Pathological_AD_Label': 50.0,
 'Per.lysosome_genes.Pathological_AD_Label': 50.0,
 'Opc.all_genes.clinical_AD_Label': 91.41935483870968,
 'Opc.m6a_genes.clinical_AD_Label': 55.74193548387097,
 'Opc.mito_genes.clinical_AD_Label': 50.0,
 'Opc.mitophagy_genes.clinical_AD_Label': 50.0,
 'Opc.AD_genes.clinical_AD_Label': 50.0,
 'Opc.lysosome_genes.clinical_AD_Label': 50.0,
 'Opc.all_genes.Pathological_AD_Label': 89.7196261682243,
 'Opc.m6a_genes.Pathological_AD_Label': 53.1860662

In [16]:
import pickle

with open('HC.pkl', 'wb') as file:
    pickle.dump(accuracy_dct, file)

print("Dictionary saved as pickle file.")

Dictionary saved as pickle file.


In [17]:
accuracy_dct

{'Per.all_genes.clinical_AD_Label': 80.76923076923077,
 'Per.m6a_genes.clinical_AD_Label': 60.256410256410255,
 'Per.mito_genes.clinical_AD_Label': 50.0,
 'Per.mitophagy_genes.clinical_AD_Label': 50.0,
 'Per.AD_genes.clinical_AD_Label': 50.0,
 'Per.lysosome_genes.clinical_AD_Label': 50.0,
 'Per.all_genes.Pathological_AD_Label': 80.76923076923077,
 'Per.m6a_genes.Pathological_AD_Label': 61.53846153846154,
 'Per.mito_genes.Pathological_AD_Label': 50.0,
 'Per.mitophagy_genes.Pathological_AD_Label': 50.0,
 'Per.AD_genes.Pathological_AD_Label': 50.0,
 'Per.lysosome_genes.Pathological_AD_Label': 50.0,
 'Opc.all_genes.clinical_AD_Label': 91.41935483870968,
 'Opc.m6a_genes.clinical_AD_Label': 55.74193548387097,
 'Opc.mito_genes.clinical_AD_Label': 50.0,
 'Opc.mitophagy_genes.clinical_AD_Label': 50.0,
 'Opc.AD_genes.clinical_AD_Label': 50.0,
 'Opc.lysosome_genes.clinical_AD_Label': 50.0,
 'Opc.all_genes.Pathological_AD_Label': 89.7196261682243,
 'Opc.m6a_genes.Pathological_AD_Label': 53.1860662

In [18]:
m6a_genes

['FTO',
 'ALKBH5',
 'METTL3',
 'METTL14',
 'WTAP',
 'KIAA1429',
 'RBM15',
 'RBM15B',
 'ZC3H13',
 'ZCCHC4',
 'METTL5',
 'METTL16',
 'CBLL1',
 'YTHDC1',
 'YTHDC2',
 'HNRNPA2B1',
 'YTHDF1',
 'YTHDF2',
 'YTHDF3',
 'IGF2BP1',
 'IGF2BP2',
 'IGF2BP3',
 'LRPPRC',
 'FMR1',
 'ELAVL1',
 'RBMX',
 'HNRNPC']

In [19]:
mito_genes

['FIS1',
 'DNM1L',
 'MFN2',
 'OPA1',
 'TOMM20',
 'PRKN',
 'AURKAIP1',
 'CHCHD1',
 'DAP3',
 'GADD45GIP1',
 'MRPL1',
 'MRPL2',
 'MRPL3',
 'MRPL4',
 'MRPL9',
 'MRPL10',
 'MRPL11',
 'MRPL12',
 'MRPL13',
 'MRPL14',
 'MRPL15',
 'MRPL16',
 'MRPL17',
 'MRPL18',
 'MRPL19',
 'MRPL20',
 'MRPL21',
 'MRPL22',
 'MRPL23',
 'MRPL24',
 'MRPL27',
 'MRPL28',
 'MRPL30',
 'MRPL32',
 'MRPL33',
 'MRPL34',
 'MRPL35',
 'MRPL36',
 'MRPL37',
 'MRPL38',
 'MRPL39',
 'MRPL40',
 'MRPL41',
 'MRPL42',
 'MRPL43',
 'MRPL44',
 'MRPL45',
 'MRPL46',
 'MRPL47',
 'MRPL48',
 'MRPL49',
 'MRPL50',
 'MRPL51',
 'MRPL52',
 'MRPL53',
 'MRPL54',
 'MRPL55',
 'MRPL57',
 'MRPL58',
 'MRPS2',
 'MRPS5',
 'MRPS6',
 'MRPS7',
 'MRPS9',
 'MRPS10',
 'MRPS11',
 'MRPS12',
 'MRPS14',
 'MRPS15',
 'MRPS16',
 'MRPS17',
 'MRPS18A',
 'MRPS18B',
 'MRPS18C',
 'MRPS21',
 'MRPS22',
 'MRPS23',
 'MRPS24',
 'MRPS25',
 'MRPS26',
 'MRPS27',
 'MRPS28',
 'MRPS30',
 'MRPS31',
 'MRPS33',
 'MRPS34',
 'MRPS35',
 'PTCD3',
 nan,
 'AGPAT4',
 'ARMC12',
 'ARMCX3',
 'BAK1